#  Movie recommendation system

Types of recommendation systems
- **Content-based recommendation system**
        - The system recommends movies based on the content or genre based on the movies or videos a user has watched. For example, if a user has watched *Iron Man*, the system will recommend movies whose content are related to the viewed movie i.e. Iron Man.
      
- **Popularity-based recommendation system**
      - The system recommend popluar movies related to the viewed movie.

- **Collaborative recommendation system**
      - The system focuses on the user's behavior or watching pattern.


This project uses **Content-based recommendation system** to recommend movies to users.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import difflib

### Data collection and preprocessing

In [5]:
data = pd.read_csv('dataset/movies.csv')

In [6]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
data.shape

(4803, 24)

#### Select features for making movie recommendations

In [8]:
film_features_ = [
    "genres", 
    "keywords",
    "tagline",
    "cast",
    "director",
]

#### Replacing null values and strings

In [9]:
# use film_features_ list to check for missing values

for feature in film_features_:
    data[feature] = data[feature].fillna("")

#### Merge all features in **film_features_** 

In [10]:
# combining all movie features
combined_features = data["genres"] + ' ' + data["keywords"] + ' ' + data["tagline"] + ' ' + data["cast"] + ' ' + data["director"]

In [11]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


#### Text encoding

Convert text data to numerical values

In [12]:
vectorizer = TfidfVectorizer()

In [13]:
_vectors = vectorizer.fit_transform(combined_features)

#### Cosine similarity

Get similarity score

In [14]:
similarity = cosine_similarity(_vectors)

In [15]:
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [16]:
similarity.shape

(4803, 4803)

### Get the name of a similar movie

In [17]:
# user input

movie_name = input('Enter a movie name: ')

In [18]:
movie_titles  = data["title"].tolist()

In [19]:
# Get a close matching name for movie name provided by the user
close_matches = difflib.get_close_matches(movie_name, movie_titles)

In [20]:
close_matches

['Superman', 'Superman II', 'Superman III']

In [21]:
close_match = close_matches[0]

In [22]:
# find the index of the movie input by the user.

movie_index = data[data["title"] == close_match]["index"].values[0]

In [23]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[movie_index]))

In [24]:
# find similarity score
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)  # lambda x: x[1]
# get the index of the movie given by the user and sort the list in descending order.

In [25]:
sorted_similar_movies

[(813, 1.0),
 (870, 0.4582408553739051),
 (2433, 0.3864273245926594),
 (14, 0.3188610536407353),
 (1296, 0.2529397869699989),
 (823, 0.21578853909502327),
 (10, 0.20133128757695565),
 (428, 0.18596641221431343),
 (72, 0.18512906308616195),
 (2793, 0.18272642077822948),
 (3, 0.18229768118495995),
 (1420, 0.1727129308212663),
 (119, 0.17180650719081145),
 (65, 0.16160582880593957),
 (3337, 0.1515876731444143),
 (4401, 0.14678890114385917),
 (4267, 0.14520436352015406),
 (1359, 0.14414128303962467),
 (210, 0.14402535832262298),
 (1238, 0.1306893940867623),
 (1282, 0.12714887083401077),
 (587, 0.1257927430818875),
 (2492, 0.1253438322333649),
 (164, 0.12497636481940003),
 (1024, 0.12419846557095111),
 (4432, 0.12366856042396873),
 (1510, 0.12144757299512393),
 (1959, 0.11861641964942257),
 (1890, 0.11744412178112948),
 (955, 0.11597960277260794),
 (1183, 0.1138043381601171),
 (1740, 0.11327489168242302),
 (634, 0.11273647980632893),
 (613, 0.11245293131098388),
 (473, 0.11152736228262713),

### Display suggested movies

In [26]:
# print names of similar movies based on the index

print("Suggested movies")
print("---"*30)

for idx, movie in enumerate(sorted_similar_movies, start=1):
    index  = movie[0]
    suggested_movie_title = data[data.index == index]["title"].values[0]

    if idx < 30:
        print(f'{idx}. {suggested_movie_title}')


Suggested movies
------------------------------------------------------------------------------------------
1. Superman
2. Superman II
3. Superman IV: The Quest for Peace
4. Man of Steel
5. Superman III
6. Crimson Tide
7. Superman Returns
8. Batman Returns
9. Suicide Squad
10. The Killer Inside Me
11. The Dark Knight Rises
12. Nanny McPhee and the Big Bang
13. Batman Begins
14. The Dark Knight
15. The Godfather
16. The Helix... Loaded
17. Batman
18. Batman
19. Batman & Robin
20. The Island of Dr. Moreau
21. The Hunting Party
22. The Abyss
23. Steel
24. Lethal Weapon 4
25. Dick Tracy
26. On the Waterfront
27. 1941
28. Star Trek IV: The Voyage Home
29. Don Juan DeMarco


## Movie recommendation system

In [29]:
user_input = input("What's your favorite movie? ")

movie_titles = data["title"].to_list()  # available movie titles

# find titles matching user_input
find_match = difflib.get_close_matches(user_input, movie_titles)

close_match = find_match[0]

get_movie_index = data[data.title == close_match]["index"].values[0]

# similarity score
similarity_score = list(enumerate(similarity[get_movie_index]))

# sort similar movies
sort_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

# print movie suggestions
print("Suggested movies")
print("---" * 30)

for idx, movie in enumerate(sort_similar_movies, start=1):
    index  = movie[0]
    suggested_movie_title = data[data.index == index]["title"].values[0]

    if idx < 30:
        print(f'{idx}. {suggested_movie_title}')

Suggested movies
------------------------------------------------------------------------------------------
1. Spider-Man
2. Spider-Man 3
3. Spider-Man 2
4. The Notebook
5. Seabiscuit
6. Clerks II
7. The Ice Storm
8. Oz: The Great and Powerful
9. Horrible Bosses
10. The Count of Monte Cristo
11. In Good Company
12. Finding Nemo
13. Clear and Present Danger
14. Brothers
15. The Good German
16. Drag Me to Hell
17. Bambi
18. The Queen
19. Charly
20. Escape from L.A.
21. Daybreakers
22. The Life Aquatic with Steve Zissou
23. Labor Day
24. Wimbledon
25. Cold Mountain
26. Hearts in Atlantis
27. Out of the Furnace
28. Bullets Over Broadway
29. The Purge: Election Year
